# Pre-processing of chSPSIM data

In [ ]:
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#import statistics
#from PIL import Image
from scipy.linalg import hadamard
import math 
from matplotlib.colors import LogNorm
from sympy.combinatorics.graycode import GrayCode
from scipy.ndimage.filters import gaussian_filter
from scipy.stats import sigmaclip
from scipy import sparse
import scipy.sparse.linalg
from scipy.sparse import linalg
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display


sys.path.append('../../fonction') 
from data_visualisation import *
from load_data import *
from matrix_tools import *
from walsh_hadamard import *
from walsh_ordered_recon import *

In [ ]:
%matplotlib

In [ ]:
def load_data(Path_files,list_files,Nl,Nh,Nc):
    Data = np.zeros((Nl,Nh,Nc))
    
    for i in range(0,2*Nh,2):
        
        Data[:,i//2] = np.float_(np.rot90(np.array(Image.open(Path_files+list_files[i]))))-np.float_(np.rot90(np.array(Image.open(Path_files+list_files[i+1]))))

    return(Data)

## Hadamard acquisition

### Data chSPSIM

In [ ]:
Dir = '../../data/2023_02_28_mRFP_DsRed_3D/Raw_data_chSPSIM_and_SPIM/data_2023_02_28/'
list_run = ['RUN0004','RUN0005','RUN0006','RUN0007','RUN0008','RUN0009','RUN0010','RUN0011','RUN0012','RUN0013','RUN0014','RUN0015','RUN0016','RUN0017','RUN0018','RUN0019','RUN0020','RUN0021','RUN0022','RUN0023','RUN0024']

Nl = 2160 # number of lines of the spectral camera
Nc = 2560 # number of columns of the spectral camera
Nh = 128  #number of patterns acquired

Bin_line = 4
Bin_lambda = 20

Nl_bin = 2048//Bin_line
Nc_bin = Nc//Bin_lambda
n = 1
for Run in list_run :
    Data_path,files = Select_data(Dir,Run)
    data = load_data(Data_path,files,Nl,Nh,Nc)
    stack = np.zeros((Nl_bin,Nh,Nc_bin))
    for i in range(Nh):
        step = bining_colonne(data[56:2104,i],Bin_lambda) # Here as we only have 2048 lines on the imaging camera we remove 56 lines at the top and the bottom of the spectrale images 
        stack[:,i] = bining_line(step,Bin_line)
    save_path = '../../data/2023_02_28_mRFP_DsRed_3D/Preprocess/'
    name = Run+'_Had_'+str(Nl_bin)+'_'+str(Nh)+'_'+str(Nc_bin)+'.npy'
    
    n += 1
    np.save(save_path+name,stack)

### Patterns

In [ ]:
Dir = '../../data/2023_02_28_mRFP_DsRed_3D/Raw_data_chSPSIM_and_SPIM/data_2023_02_28/'
Run = 'RUN0002'
Data_path,files = Select_data(Dir,Run)

In [ ]:
%%time
Nl = 2048 # number of lines of the imaging camera
Nc = 2048 # number of columns of the imaging camera
Nh = 128#64# #number of patterns acquired
Bin_col = Bin_line
data = load_data(Data_path,files,Nl,Nh,Nc)

mean pattern around the middle of the field of view

In [ ]:
motifs = np.zeros((Nh,Nc))
for i in range(Nh):
    motifs[i] = np.sum(data[1000:1048,i],0)
motifs = bining_colonne(motifs,Bin_col)
plot_im2D(motifs)

In [ ]:
save_path = '../../data/2023_02_28_mRFP_DsRed_3D/Reconstruction/Mat_rc/'
name = "motifs_Hadamard_"+str(Nh)+"_"+str(Nc//Bin_col)+".npy"
np.save(save_path+name,motifs)